In [153]:
import tensorflow.keras.backend as K
import pandas as pd
from gensim.models.keyedvectors import KeyedVectors
from keras.models import Sequential, model_from_json
from keras.layers.core import Dropout, Dense, Activation, Flatten, Reshape
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.utils import np_utils
from collections import defaultdict
from sklearn import preprocessing

In [1]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove2word2vec(glove_input_file="embeddings/glove/glove.twitter.27B.200d.txt", word2vec_output_file="embeddings/glove/gensim_glove.twitter.27B.200d.txt")

(1193517, 200)

In [2]:
glove_model = KeyedVectors.load_word2vec_format("embeddings/glove/gensim_glove.twitter.27B.200d.txt", binary=False)

In [157]:
with open('data/train.jsonl') as f:
    train_data = pd.read_json(f, lines=True)
with open('data/test.jsonl') as f:
    test_data = pd.read_json(f, lines=True)

In [164]:
le = preprocessing.LabelEncoder()
y = le.fit_transform(train_data['label'])
y_train = np_utils.to_categorical(y, num_classes=2)

In [159]:
train_data['context'] = train_data['context'].str.join(' ')

In [160]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

MAX_NB_WORDS = 10000
MAX_SEQUENCE_LENGTH = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(train_data['response'])
tokenizer.fit_on_texts(train_data['context'])
tokenizer.fit_on_texts(test_data['response'])
tokenizer.fit_on_texts(test_data['context'])
X_train_resp = pad_sequences(tokenizer.texts_to_sequences(train_data['response']), maxlen=MAX_SEQUENCE_LENGTH)
X_train_cont = pad_sequences(tokenizer.texts_to_sequences(train_data['context']), maxlen=MAX_SEQUENCE_LENGTH)
X_test_resp = pad_sequences(tokenizer.texts_to_sequences(test_data['response']), maxlen=MAX_SEQUENCE_LENGTH)
X_test_cont = pad_sequences(tokenizer.texts_to_sequences(test_data['context']), maxlen=MAX_SEQUENCE_LENGTH)

In [161]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 38565 unique tokens.


In [162]:
EMBEDDING_DIM = 200

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    if word in glove_model.vocab:
        embedding_matrix[i] = glove_model[word]

In [163]:
#https://github.com/AniSkywalker/SarcasmDetection/blob/master/src/sarcasm_detection_model_CNN_LSTM_DNN_word2vec.py
hidden_units=256

model = Sequential()
model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH, trainable=False))

# model.add(Reshape((maxlen, emb_weights.shape[1], 1)))

model.add(Convolution1D(emb_weights.shape[1], 3, kernel_initializer='he_normal', padding='valid',
                        activation='sigmoid',
                        input_shape=(1, MAX_SEQUENCE_LENGTH)))
# model.add(MaxPooling1D(pool_size=3))

model.add(Convolution1D(emb_weights.shape[1], 3, kernel_initializer='he_normal', padding='valid',
                        activation='sigmoid',
                        input_shape=(1, MAX_SEQUENCE_LENGTH - 2)))
# model.add(MaxPooling1D(pool_size=3))

model.add(Dropout(0.25))

model.add(LSTM(hidden_units, kernel_initializer='he_normal', activation='sigmoid', dropout=0.5,
               return_sequences=True))
model.add(LSTM(hidden_units, kernel_initializer='he_normal', activation='sigmoid', dropout=0.5))

model.add(Dense(hidden_units, kernel_initializer='he_normal', activation='sigmoid'))
model.add(Dense(2, activation='softmax'))
adam = Adam(lr=0.0001)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['acc'])
print('No of parameter:', model.count_params())

print(model.summary())

No of parameter: 9013186
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 100, 200)          7713200   
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 98, 200)           120200    
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 96, 200)           120200    
_________________________________________________________________
dropout_2 (Dropout)          (None, 96, 200)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 96, 256)           467968    
_________________________________________________________________
lstm_5 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense_4 (Dense)              

In [165]:
modelCheckpoint = ModelCheckpoint('best_sarcasm_model.hdf5', save_best_only = True)
model.fit(X_train_resp, y_train, batch_size=8, epochs=100, validation_split=0.2 , shuffle=True, callbacks=[modelCheckpoint])

Epoch 1/100
500/500 [==============================] - 46s 91ms/step - loss: 0.6774 - acc: 0.6140 - val_loss: 1.0001 - val_acc: 0.0000e+00
Epoch 2/100
500/500 [==============================] - 48s 97ms/step - loss: 0.6638 - acc: 0.6242 - val_loss: 1.2088 - val_acc: 0.0000e+00
Epoch 3/100
500/500 [==============================] - 55s 110ms/step - loss: 0.6602 - acc: 0.6240 - val_loss: 1.1739 - val_acc: 0.0000e+00
Epoch 4/100
500/500 [==============================] - 51s 103ms/step - loss: 0.6051 - acc: 0.6775 - val_loss: 1.0085 - val_acc: 0.4040
Epoch 5/100
500/500 [==============================] - 51s 101ms/step - loss: 0.5414 - acc: 0.7352 - val_loss: 1.0116 - val_acc: 0.4360
Epoch 6/100
500/500 [==============================] - 50s 100ms/step - loss: 0.5167 - acc: 0.7477 - val_loss: 0.7914 - val_acc: 0.5500
Epoch 7/100
500/500 [==============================] - 49s 97ms/step - loss: 0.5206 - acc: 0.7495 - val_loss: 1.1835 - val_acc: 0.3790
Epoch 8/100
500/500 [==================

Epoch 61/100
500/500 [==============================] - 57s 113ms/step - loss: 0.3510 - acc: 0.8580 - val_loss: 1.0650 - val_acc: 0.5200
Epoch 62/100
500/500 [==============================] - 49s 97ms/step - loss: 0.3505 - acc: 0.8635 - val_loss: 0.9675 - val_acc: 0.5570
Epoch 63/100
500/500 [==============================] - 48s 96ms/step - loss: 0.3511 - acc: 0.8597 - val_loss: 1.1498 - val_acc: 0.4930
Epoch 64/100
500/500 [==============================] - 48s 96ms/step - loss: 0.3430 - acc: 0.8620 - val_loss: 1.3971 - val_acc: 0.4020
Epoch 65/100
500/500 [==============================] - 51s 102ms/step - loss: 0.3442 - acc: 0.8668 - val_loss: 1.2241 - val_acc: 0.4420
Epoch 66/100
500/500 [==============================] - 48s 96ms/step - loss: 0.3389 - acc: 0.8622 - val_loss: 1.1675 - val_acc: 0.4860
Epoch 67/100
500/500 [==============================] - 47s 95ms/step - loss: 0.3413 - acc: 0.8685 - val_loss: 1.1922 - val_acc: 0.4660
Epoch 68/100
500/500 [========================